In [ ]:
# lfsr_lab.py

class LFSR:
    """
    n-bit Fibonacci LFSR.
    - state is an integer in [1, 2^n - 1] (we'll avoid 0 because it locks)
    - taps is a list of bit positions (0 = LSB, n-1 = MSB) used to compute feedback.
    """
    def __init__(self, n, state, taps):
        assert 1 <= state < (1 << n)
        self.n = n
        self.state = state
        self.taps = taps

    def step(self):
        """
        Returns one output bit (0/1), updates internal state by 1 clock.
        Convention:
          - output bit = LSB of current state
          - feedback bit = XOR of tapped bits of current state
          - shift right by 1, insert feedback at MSB
        """
        least_significant_bit = self.state & 1
        sum_taps=0
        for t in self.taps:
            tap=(self.state>>t)&1
            sum_taps=tap^sum_taps
        self.state=self.state>>1
        sum_taps=sum_taps<<(self.n-1)
        self.state=self.state+sum_taps
        return least_significant_bit

    def bits(self, k):
        return [self.step() for _ in range(k)]
    
class CSSKeystream:
    def __init__(self, lfsr17, lfsr25):
        self.lfsr17 = lfsr17
        self.lfsr25 = lfsr25
        self.carry = 0

    def next_byte(self):
        """
        Produce one keystream byte using:
        - 8 bits from 17-bit LFSR
        - 8 bits from 25-bit LFSR
        - carry from previous round
        """
        # TODO 1: collect 8 bits from lfsr17 into a byte A
        # TODO 2: collect 8 bits from lfsr25 into a byte B
        # TODO 3: add A + B + carry
        # TODO 4: update carry
        # TODO 5: return byte (0..255)
        #print(f"{A:08b}")
        A=self._collect_bits(self.lfsr17.bits(8))
        B=self._collect_bits(self.lfsr25.bits(8))
        self.carry=

    def _collect_bits(self, bits):
        Z=0
        for b in bits:
            Z=(Z<<1)+b
        return Z

        

def test_keystream_runs():
    l17 = LFSR(17, state=0b10101010101010101, taps=[0, 3])
    l25 = LFSR(25, state=0b1110001110001110001110001, taps=[0, 2, 5])

    ks = CSSKeystream(l17, l25)
    stream = [ks.next_byte() for _ in range(20)]

    print(stream)
    assert all(0 <= b <= 255 for b in stream)
    print("✓ keystream generator works")

test_keystream_runs()

01010101
01010101
11111111
01111111
00000001
10100000
00000010
11101000
00000101
10101010
10001010
11111110
10110111
00000010
11000010
10000101
11110100
00101010
01010100
01011111
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


TypeError: '<=' not supported between instances of 'int' and 'NoneType'

In [3]:
def test_smoke():
    l = LFSR(n=4, state=0b1111, taps=[0, 1])
    bits = l.bits(20)

    print("Bits:", bits)

    assert all(b in (0, 1) for b in bits)
    print("✓ smoke test passed")
test_smoke()

Bits: [1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0]
✓ smoke test passed


In [4]:
def test_state_width():
    n = 5
    l = LFSR(n=n, state=0b10101, taps=[0, 2])

    for _ in range(100):
        l.step()
        assert 1 <= l.state < (1 << n)

    print("✓ state width test passed")

test_state_width()

✓ state width test passed


In [5]:
def test_determinism():
    l1 = LFSR(6, state=0b100111, taps=[0, 1])
    l2 = LFSR(6, state=0b100111, taps=[0, 1])

    bits1 = l1.bits(50)
    bits2 = l2.bits(50)

    assert bits1 == bits2
    print("✓ determinism test passed")

test_determinism()

✓ determinism test passed


In [6]:
def test_zero_state_rejected():
    try:
        LFSR(n=5, state=0, taps=[0, 2])
    except AssertionError:
        print("✓ zero state correctly rejected")
    else:
        raise AssertionError("zero state was NOT rejected")
test_zero_state_rejected()

✓ zero state correctly rejected


In [7]:
def test_not_trivial_period():
    l = LFSR(5, state=0b10001, taps=[0, 2])
    seen = set()

    for _ in range(100):
        if l.state in seen:
            break
        seen.add(l.state)
        l.step()

    print("Visited states:", len(seen))
    assert len(seen) > 5
    print("✓ non-trivial period")

test_not_trivial_period()

Visited states: 31
✓ non-trivial period
